## PROJET HADOOP - MS-SIO-2019 - SNCF - API TRANSILIEN - PARTIE I

#### SPARK STRUCTURED STREAMING (KAFKA CONSUMER)

P. Hamy,  N. Leclercq, L. Poncet - MS-SIO-2019

In [ ]:
import os
import json
import time
from pyspark.sql import SparkSession
import pyspark.sql.types as st
import pyspark.sql.functions as sf

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.ERROR, datefmt='%H:%M:%S')

Changement du logging level afin d'éliminer le bruit généré dans la console par un [_warning_](https://stackoverflow.com/questions/39351690/got-interruptedexception-while-executing-word-count-mapreduce-job) récurrent

In [ ]:
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

### QUESTION 1.1 : calcul et publication du temps d'attente moyen par station

Cette section donne le détail de construction du flux de données relatif à la dernière heure - tranche horaire à laquelle les métriques demandées s'appliquent. Le code est segmenté afin d'en faliciter le commentaire. Il sera repris plus loin afin d'être encapsulé dans une classe.  

#### Création de la session Spark associé au flux Kafka

In [ ]:
kafka_session = SparkSession.builder.appName("MS-SIO-HADOOP-PROJECT-KAFKA-STREAM").getOrCreate()

Limitation du nombre de taches lancées par spark (conseil de configutation glané sur internet pour les configurations matérielles les plus modestes).

In [ ]:
kafka_session.conf.set('spark.sql.shuffle.partitions', 4)

#### Création du flux Kafka
On utilise ici un [structured spark stream](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html) associé à une source Kafka. 

Il s'agit de spécifier la source via l'adresse du serveur Kafka et le nom du topic auquel on souhaite s'abonner. 

In [ ]:
kafka_stream = kafka_session \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "sandbox-hdp.hortonworks.com:6667") \
    .option("subscribe", "transilien-02") \
    .option("startingOffsets", "earliest") \
    .load()

#### Schéma de désérialisation des messages  
Les messages injectés dans le flux Kafka sont sérialisés et encodés en binaire dans le champ _value_ du dataframe (format générique des dataframe issus d'un stream Kafka).
```
kafka_stream.printSchema()
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 ```
Il est donc nécessaire despécifier le schéma de désérialisation qui sera passé à la fonction **from_json**.

In [ ]:
json_schema = st.StructType(
    [
        st.StructField("station", st.IntegerType(), True),
        st.StructField("train", st.StringType(), True),
        st.StructField("timestamp", st.TimestampType(), True),
        st.StructField("mode", st.StringType(), True)
    ]
)

A travers, la variable **json_options**, on précise également le format du champ _timestamp_ afin que les valeurs temporelles soient correctement interprétées.

In [ ]:
json_options = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

#### Séquence de calcul du temps d'attente moyen par station : étape-01 
Désérialisation/reformatage des messages.

In [ ]:
df = kafka_stream \
    .select(sf.from_json(sf.col("value").cast("string"), json_schema, json_options).alias("departure")) \
    .select("departure.*")

A l'issue de opération le dataframe a le schéma suivant:
```
df.printSchema()
root
 |-- station: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- train: string (nullable = true)
```
#### Séquence de calcul du temps d'attente moyen par station : étape-02
Spécification de la watermark du stream spark. Nous n'acceptons pas les messages ayant plus d'une minute de retard. Il s'agit d'un choix arbitraire qui n'a que peu d'intérêt dans notre projet. Il est toutefois nécessaire de spécifier cette valeur car l'implémentation sous-jacente doit borner l'accumulation des données du stream. [La documentation de Spark explique clairement le concept de _Stateful Stream Processing in Structured Streaming_](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html).

In [ ]:
df = df.withWatermark("timestamp", "1 minutes")

#### Séquence de calcul du temps d'attente moyen par station : étape-03
Un train apparaitra dans les réponses aux requêtes de l'API SNCF tant que son heure de départ n'appartient pas au passé. On supprime donc les doublons associés aux couples (train, heure de départ). Inutile d'ajouter la station à la contrainte d'exclusion car l'idenfiant d'un train est unique.

In [ ]:
df = df.dropDuplicates(["train", "timestamp"])

#### Séquence de calcul du temps d'attente moyen par station : étape-04
Définition de la fênêtre (temporelle) dans laquelle les calculs sont aggrégés. Il s'agit ici de calculer le temps d'attente moyen par station **sur la dernière heure**. Notre fenêtre a donc une largeur temporelle de 60 minutes (_window length_). On choisit de suivre cette moyenne par pas de 2 minutes (_sliding interval_). Dans la mesure où le calcul est demandé par station, la fonction _groupBy_ s'applique au champ _station_ du dataframe.

In [ ]:
df = df.groupBy("station", sf.window("timestamp", "60 minutes", "2 minutes"))

#### Séquence de calcul du temps d'attente moyen par station : étape-05
Pour chaque station, on définit le temps moyen d'attente sur une période de P minutes comme le rapport de P vs le nombre de trains au départ de cette station sur la période P. Ici P = 60 minutes. 

On crée une _aggrégation_ qui contiendra, pour chaque station et pour chaque fenêtre d'une heure :
- une colonne _nt_ qui indiquant le nombre de trains sur la période
- une colonne _awt_ donnant le temps d'attente moyen recherché. 

La colonne _nt_ est injectée à titre indicatif (visualisation dans la console, validation du calcul).

In [ ]:
df = df.agg(sf.count("train").alias("nt"), sf.format_number(60./ sf.count("train"), 2).cast("double").alias("awt"))

#### Séquence de calcul du temps d'attente moyen par station : étape-06
Selection de la fenêtre temporelle associée à la dernière heure. 

Il s'agit de selectionner, parmis les N fenêtres temporelles produites par Spark, celle qui correspond à la dernière heure écoulée. On utilise ici la fonction **current_timestamp** de Spark afin de rendre la sélection dynamique (i.e. glissante). Le calcul est effectué est dans l'unité de **unix_timestamp** (la seconde) - beaucoup plus facile à manipulée dans ce contexte. 

Dans l'idée de pourvoir visualiser (si besoin) les valeurs mises en jeu dans le calcul, on choisit de créér les colonnes associées:
- oha = one hour ago = now - 62 minutes = now - (window length + sliding interval) 
- now = now - 1 minutes = now - (sliding interval / 2.) => **valeur ajustée pour n'obtenir qu'une seule fenêtre**
- wstart = window.start = borne inférieure de la fenêtre temporelle 
- wend = window.wend = borne supérieure de la fenêtre temporelle

**La clause _where_ permet de selectionner la fenêtre associée à la dernière heure**.  

In [ ]:
df = df \
    .withColumn("oha", sf.unix_timestamp(sf.current_timestamp()) - int((60 + 2) * 60)) \
    .withColumn("now", sf.unix_timestamp(sf.current_timestamp()) - int(60 * 2) / 2.) \
    .withColumn("wstart", sf.unix_timestamp("window.start")) \
    .withColumn("wend", sf.unix_timestamp("window.end")) \
    .where((sf.col("oha") <= sf.col("wstart")) & (sf.col("wend") <= sf.col("now")))

A ce stade, _df_ constitue 'l'état' de référence de notre stream Spark. C'est à partir de cet état que l'on produit les résultats, métriques, indicateurs, ... demandés

#### Validation des étapes 01 à 06

In [ ]:
query = df \
    .select("station", "window", "nt", "awt") \
    .writeStream \
    .format("console") \
    .option("truncate", False) \
    .outputMode("complete") \
    .start()

````
+--------+------------------------------------------+---+----+
|station |window                                    |nt |awt |
+--------+------------------------------------------+---+----+
|87382473|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87386425|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87382259|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|19 |3.16|
|87382457|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|18 |3.33|
|87382440|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|20 |3.0 |
|87382333|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|14 |4.29|
|87382887|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|12 |5.0 |
|87334482|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|30 |2.0 |
|87381137|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|48 |1.25|
|87386318|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|17 |3.53|
|87382374|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|12 |5.0 |
|87382499|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|27 |2.22|
|87382655|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|5  |12.0|
|87382382|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|25 |2.4 |
|87381905|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|31 |1.94|
|87384008|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|16 |3.75|
|87386003|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|19 |3.16|
|87386300|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|24 |2.5 |
|87381129|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|38 |1.58|
|87386409|[2019-02-28 08:06:00, 2019-02-28 09:06:00]|24 |2.5 |
+--------+------------------------------------------+---+----+
```

Arrêt de la query

In [ ]:
query.stop()

In [ ]:
kafka_session.streams.active

### QUESTION 1.2 à 1.6
- Calcul du temps moyen d’attente sur la ligne station par station sur la dernière heure
- Calcul du temps moyen d’attente globale sur la ligne sur la dernière heure
- Trier les stations par temps d’attente moyen sur la dernière heure
- Trouver la station avec le temps d’attente le plus élevée sur la dernière heure
- Trouver la station avec le temps d’attente le moins élevée sur la dernière heure
- Construire un tableau de bord dans Tableau Software sur la base de ces indicateurs

Les calculs demandés nécessiteraient une seconde opération aggrégation sur le stream (last_hour_stream). Or, en l'état actuel de Spark (2.4), [il n'est pas possible d'enchainer plusieurs opération d'aggrégation sur un même stream](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#unsupported-operations). Il nous faut donc trouver une solution de contournement.

L'idée retenue est d'effectuer les calculs sur chaque batch de last_hour_stream. Cette approche fonctionne ici car chaque batch contient l'intégralité des données sur lesquelles les calculs doivent être réalisés pour produire les résultats attendus - i.e. les données d'attente moyenne par station. Ces résulats sont enregistrés sous _Hive_ dans des tables spécifiques. Ils sont ainsi rendus accéssibles depuis Tableau pour l'élaboration du tableau de bord. 

Les calculs (Q1.2 à Q.1.5) sont regroupés dans un callback du type _foreachBatch_ dont les appels sont déclenchés par une _StreamingQuery_.

La classe _TransilienStreamProcessor_ implémente les fonctionnalités demandées.

In [ ]:
class TransilienStreamProcessor():
    
    # -------------------------------------------------------------------------------
    def __init__(self, config):
    # -------------------------------------------------------------------------------
        # store configuration 
        self.config = config
        
        # setup logger
        self.logger = logging.getLogger("TSP")
        self.logger.setLevel(logging.DEBUG if self.config['verbose'] else logging.ERROR)
        self.logger.debug("TSP: initializing...")
        
        # kafka oriented spark session (configured to process incoming Kafka messages)
        self.logger.debug(f"TSP: creating Kafka-SparkSession")
        self.kafka_session = SparkSession.builder.appName("MS-SIO-HADOOP-PROJECT-STREAM").getOrCreate()
        self.kafka_session.conf.set("spark.sql.shuffle.partitions", self.config['spark_sql_shuffle_partitions'])
        self.logger.debug(f"TSP: initializing Kafka stream")
        self.kafka_stream = self.setup_kafka_stream()
        self.logger.debug(f"`-> done!")
        
        # hive oriented spark session (configured to save the results at the right place in Hive)
        self.logger.debug(f"TSP: creating Hive-SparkSession")
        self.hive_session = SparkSession \
            .builder \
            .master("yarn") \
            .appName("MS-SIO-HADOOP-PROJECT-PROCESS") \
            .config("spark.sql.warehouse.dir", self.config['hive_warehouse']) \
            .config("hive.metastore.uris", self.config['hive_metastore']) \
            .enableHiveSupport() \
            .getOrCreate() 
        self.hive_session.conf.set("spark.sql.shuffle.partitions", self.config['spark_sql_shuffle_partitions'])
        self.logger.debug(f"`-> done!")
        
        # tables in which the produced data will be published
        self.tables = [
            # Q1.1 & Q1.3: (ordered) average waiting time in minutes per station (on the last hour period)
            "ordered_awt", 
            # Q1.2: global average waiting time in minutes (on the last hour period) 
            "global_awt", 
            # Q1.4: min average waiting time in minutes (on the last hour period) 
            "min_awt", 
            # Q1.5: max average waiting time in minutes (on the last hour period) 
            "max_awt"
        ]
        
        # check/create the database 
        self.__select_database()
        
        # drop existing tables (optional)
        if self.config['drop_existing_tables']:
            self.__drop_tables()
            
        # the 'save as table' streaming query - acts as a trigger for computeAndSaveAsTables
        self.sat_query = None
        
        # the 'console' streaming query - print batches in the console  
        self.csl_query = None
        
        # processing time (i.e. streaming queries trigger period)
        self.processing_time = "15 seconds" #"1 minutes"
        
        # start the streaming query?
        if self.config['auto_start']:
            self.start()
            
        self.logger.debug(f"initialization done!")

    # -------------------------------------------------------------------------------
    def __select_database(self):
    # -------------------------------------------------------------------------------
        self.logger.debug(f"TSP: check '{config['hive_database']}' database exists (will be created otherwise)")
        self.hive_session.sql(f'create database if not exists {config["hive_database"]} location "{config["hive_warehouse"]}"')                  
        self.logger.debug(f"`-> done!")           
        self.logger.debug(f"TSP: using '{config['hive_database']}' database")
        self.hive_session.sql(f'use  {config["hive_database"]}')
        self.logger.debug(f"`-> done!")           

    # -------------------------------------------------------------------------------
    def __drop_tables(self):
    # -------------------------------------------------------------------------------
        # drop existing tables (best effort impl.)
        for table in self.tables:
            try:
                self.logger.debug(f"TSP: dropping table {table}")
                self.session.sql(f"drop table transilien.{table}")
                self.logger.debug(f"`-> done!")
            except Exception as e:
                pass
    
    # -------------------------------------------------------------------------------
    def setup_kafka_stream(self):
    # -------------------------------------------------------------------------------
        wm = float(self.config['kafka_stream_watermark'])
        wl = float(self.config['kafka_stream_window_length'])
        si = float(self.config['kafka_stream_sliding_interval'])
        oha_offset = int((wl + si) * 60.)
        now_offset = int(60. * si / 2.)
        # setup 'last hour stream'
        return self.kafka_session \
            .readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", self.config['kafka_broker']) \
            .option("subscribe", self.config['kafka_topic']) \
            .option("startingOffsets", "earliest") \
            .load() \
            .select(sf.from_json(sf.col("value").cast("string"), self.config['json_schema'], self.config['json_options']).alias("departure")) \
            .select("departure.*") \
            .withWatermark("timestamp", f"{int(wm)} minutes") \
            .dropDuplicates(["train", "timestamp"]) \
            .groupBy("station", sf.window("timestamp", f"{int(wl)} minutes", f"{int(si)} minutes")) \
            .agg(sf.count("train").alias("nt"), sf.format_number(wl / sf.count("train"), 2).cast("double").alias("awt")) \
            .withColumn("oha", sf.unix_timestamp(sf.current_timestamp()) - oha_offset) \
            .withColumn("now", sf.unix_timestamp(sf.current_timestamp()) - now_offset) \
            .withColumn("wstart", sf.unix_timestamp("window.start")) \
            .withColumn("wend", sf.unix_timestamp("window.end")) \
            .where((sf.col("oha") <= sf.col("wstart")) & (sf.col("wend") <= sf.col("now"))) \
            .drop("oha", "now", "wstart", "wend")
        
    # -------------------------------------------------------------------------------
    def start(self):
    # -------------------------------------------------------------------------------
        # stop the streaming query if already running
        self.stop()
        # create then start the 'save' streaming query on the specified streaming dataframe
        # also make 'self.computeAndSaveAsTables' member function the associated 'foreachBatch' callback
        self.logger.debug(f"TSP: starting 'SaveAsTables' streaming query")
        self.sat_query =  self.kafka_stream \
                            .writeStream \
                            .trigger(processingTime=self.processing_time) \
                            .foreachBatch(self.computeAndSaveAsTables) \
                            .outputMode("complete") \
                            .start() 
        self.logger.debug(f"`-> done!")
        # create then start the 'console' streaming query on the specified streaming dataframe
        self.logger.debug(f"TSP: starting 'Console' streaming query")
        self.csl_query =  self.kafka_stream \
                            .orderBy("awt") \
                            .writeStream \
                            .trigger(processingTime=self.processing_time) \
                            .outputMode("complete") \
                            .format("console") \
                            .option("truncate", False) \
                            .start() 
        self.logger.debug(f"`-> done!")
        self.logger.debug(f"TSP: streaming queries are running")
      
    # -------------------------------------------------------------------------------
    def stop(self):
    # -------------------------------------------------------------------------------
        # stop the streaming query (best effort impl.)
        if self.sat_query is  not None:
            try:
                self.logger.debug(f"TSP: stopping 'SaveAsTables' streaming query")
                self.sat_query.stop()
            except Exception as e:
                pass
            finally:
                self.sat_query = None
                self.logger.debug(f"`-> done!")
        if self.csl_query is  not None:
            try:
                self.logger.debug(f"TSP: stopping 'Console' streaming query")
                self.csl_query.stop()
            except Exception as e:
                pass
            finally:
                self.csl_query = None
                self.logger.debug(f"`-> done!")
       
    # -------------------------------------------------------------------------------
    def cleanup(self):
    # -------------------------------------------------------------------------------
        # cleanup the underlying session 
        # TODO: not sure this is the right way to do the job
        self.logger.debug(f"TSP: shutting down Kafka-SparkSession")
        self.kafka_session.stop()
        self.logger.debug(f"`-> done!")
        self.logger.debug(f"TSP: shutting down Hive-SparkSession")
        self.hive_session.sql("clear cache")
        self.hive_session.stop()
        self.logger.debug(f"`-> done!")
      
    # -------------------------------------------------------------------------------
    def turnVerboseOn(self):
    # -------------------------------------------------------------------------------
        # turn verbose on
        self.logger.setLevel(logging.DEBUG)
       
    # -------------------------------------------------------------------------------
    def turnVerboseOff(self):
    # -------------------------------------------------------------------------------
         # turn verbose off
        self.logger.setLevel(logging.ERROR)
        
    # -------------------------------------------------------------------------------
    def computeAndSaveAsTables(self, batch, batch_number):
    # -------------------------------------------------------------------------------
        # the big trick... heart of the functionnality implemented by this class
        try:
            # be sure we have some data to handle (incoming dataframe not empty)
            # this will avoid creating empty tables on Hive side 
            if batch.rdd.isEmpty():
                self.logger.warning(f"TSP: ignoring empty batch #{batch_number}")
                return
            # compute requested data - don't change computation order (list order matters)
            results = []
            # Q1.1 & Q1.3: compute ordered average waiting time in minutes (on the last hour period)
            results.append(batch.orderBy(sf.asc("awt")).select(sf.col("station"), sf.col("awt")))
            # Q1.2: compute global average waiting time in minutes (on the last hour period) 
            results.append(batch.agg(sf.count("station").alias("number_of_stations"), sf.avg("awt").alias("global_awt")))
            # Q1.4: compute min average waiting time in minutes (on the last hour period)
            results.append(batch.orderBy(sf.asc("awt")).limit(1).select(sf.col("station"), sf.col("awt").alias("min_awt")))
            # Q1.5: compute max average waiting time in minutes (on the last hour period)
            results.append(batch.orderBy(sf.desc("awt")).limit(1).select(sf.col("station"), sf.col("awt").alias("max_awt")))
            # Q1.6: save results as tables so that Tableau Software can access the data
            for dataframe, table in zip(results, self.tables):
                if dataframe.rdd.isEmpty(): # avoid creating empty tables on Hive side
                    continue
                self.logger.debug(f"saving transilien.{table} for batch #{batch_number}")
                if self.logger.getEffectiveLevel() == logging.DEBUG:
                    dataframe.show(3, False)
                t = time.time()
                dataframe.write.mode("overwrite").saveAsTable(f"transilien.{table}")
                self.logger.debug(f"`-> took {round(time.time() - t, 2)} s" )
            self.logger.debug(f"computeAndSaveAsTables successfully executed for batch #{batch_number}")
        except Exception as e:
            self.logger.error(f"TSP: failed to update Hive tables for batch #{batch_number}")
            self.logger.error(e)

Pour mémoire : dans un contexte de production, le code qui précède serait injecté dans un script Python lancé via _spark-submit_. Dans un tel cas, serait nécessaire placer un appel à _awaitTermination_ sur chaque streaming query.

In [ ]:
# configuration parameters
config = {}

# json schema & options for kafka messages deserialization 
config['json_schema'] = st.StructType(
    [
        st.StructField("station", st.IntegerType(), True),
        st.StructField("train", st.StringType(), True),
        st.StructField("timestamp", st.TimestampType(), True),
        st.StructField("mode", st.StringType(), True)
    ]
)
config['json_options'] = {"timestampFormat": "yyyy-MM-dd'T'HH:mm:ss.sss'Z'"}

# spark sesssions options
config['spark_sql_shuffle_partitions'] = 4

# kafka source configuration: broker & topic
config['kafka_broker'] = "sandbox-hdp.hortonworks.com:6667"
config['kafka_topic'] = "transilien-02"

# kafka stream configuration: structured stream windiowing
config['kafka_stream_watermark'] = 1 
config['kafka_stream_window_length'] = 60
config['kafka_stream_sliding_interval'] = 2

# hive session configuration
config['hive_database'] = "transilien"
config['hive_warehouse'] = "hdfs://sandbox-hdp.hortonworks.com:8020/api-transilien"
config['hive_metastore'] = "thrift://sandbox-hdp.hortonworks.com:9083"

# misc. options
config['drop_existing_tables'] = False 
config['auto_start'] = False 
config['verbose'] = True
config['context'] = "jupyspark"

In [ ]:
ti = TransilienStreamProcessor(config)

In [ ]:
ti.start()

In [ ]:
ti.turnVerboseOff()

In [ ]:
ti.turnVerboseOn()

In [ ]:
ti.stop()

Cleanup...
NB: the python kernel must be restart after a call to TransilienStreamProcessor.cleanup 

In [ ]:
ti.cleanup()